In [ ]:
import numpy as np
import os
import sys

%matplotlib widget
import matplotlib.pyplot as plt
import scipy.io as scio
import asdprocessor

import matplotlib as mpl
from cycler import cycler
import csv

In [ ]:
#Define inputs 
data_dir = '/Users/reckert/Documents/Data/'
out_dir  = data_dir + 'output/' 
fig_dir  = out_dir + 'figures/'

print('Output directory: {}'.format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

print('Figure directory: {}'.format(fig_dir))
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

In [ ]:
#Load in asd files and process
file_tag_list = asdprocessor.get_asd_filetags(data_dir) #Get unique filetags and sort them

print(f'Processing .asd files begining with tags:\n{file_tag_list}')
n_spectra_used = np.zeros(len(file_tag_list))
n_outliers = n_spectra_used.copy()

for ii in range(len(file_tag_list)):
    # List all files in data_dir with given file_tag
    file_tag = file_tag_list[ii]
    print(file_tag)

    proc = asdprocessor.processor(flag_treat_REF_as_RAW = False)
    # Default input values: 
    # known_ref_idx = 0             # Index of known white reference measurement if measured RAW or flat_treat_REF_as_RAW = True
    # flag_treat_REF_as_RAW = False # True if measured in white reference mode & also have separate measurements of white reference you want to find
    # spectralon_data_path = './correction_data/spectralon_rfl.txt' # Location of spectralon data
    # jump_corr_method = 'hueni' # 'hueni' or 'asdparabolic'
    # jump_corr_iterations = 5   # Used for 'hueni' method only

    # Read in all files associated with a certain file_tag
    proc.load_asd_files(data_dir,file_tag)
    # Returns: spectra_mat and reference_mat (loaded DN spectra and references)

    # Identify white references, identify outliers, interpolate between them if desired
    proc.process_white_references(interp_method='linear') #Could be folded into get_reflectance
    # Default input values:
    # interp_method = 'linear' # Linear interpolation; other options are any accepted by scipy.interpolate.interp1d
    # Returns: interpolated white references for identified measurements

    # Reflectance based on white references in step above
    proc.calculate_reflectances()
    # Returns: identified measurement reflectances

    #Account for spectralon reflectance 
    proc.apply_spectralon_rfl_corr()

    #Account for spectralon being at 45 deg to measurement angle
    proc.apply_spectralon_45deg_corr()

    #Spectral jump correction at 1000-1001 nm and 1800-1801 nm
    proc.apply_jump_correction(jump_corr_method='asdparabolic')
    # Inputs for jump_corr_method and jump_corr_iterations here overrides those input when proc object is created

    #Find reflectance outliers
    proc.calculate_outliers()
    # By default, finds outliers that are sigma_factor*stddev greater/less than the mean
    # and outliers that are less than cossim_threshold cosine similarity to the mean
    # (Needs to be edited for passing more control to the user)
    # Input defaults: std_sigma_factor = 3, cossim_threshold = 0.985
    

    #Get final mean, std, and the reflectances that were used to create the mean, std
    rfl_mean, rfl_std, rfl_mat = proc.calculate_final_reflectance()

    #Save the measurements in an array for comparison
    if ii == 0:
        mean_spectra_mat = np.zeros((len(file_tag_list),proc.n_channels))
        std_spectra_mat = np.zeros((len(file_tag_list),proc.n_channels))

    mean_spectra_mat[ii,:] = rfl_mean
    std_spectra_mat[ii,:]  = rfl_std


In [ ]:
# Plot of mean spectra
plt.figure()
plt.plot(proc.wavelengths,np.transpose(mean_spectra_mat))   
plt.title('Mean reflectance spectra')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')

# Put a legend to the right of the current axis
ax = plt.gca()
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.6, box.height])
ax.legend(file_tag_list,loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Plot of final spectrum's constituants & mean +/- 2*stddev
mpl.rcParams['axes.prop_cycle'] = cycler(color=mpl.cm.rainbow(np.linspace(0,1,num=rfl_mat.shape[0])))

line_width = 3
plt.figure()
plt.plot(proc.wavelengths,np.transpose(rfl_mat),linewidth=line_width/2)
plt.plot(proc.wavelengths,np.squeeze(rfl_mean),'k',linewidth=line_width)
plt.plot(proc.wavelengths,np.squeeze(rfl_mean-2*rfl_std),'0.4',linewidth=line_width/2,linestyle='-.')     
plt.plot(proc.wavelengths,np.squeeze(rfl_mean+2*rfl_std),'0.4',linewidth=line_width/2,linestyle='-.') 
plt.title(f'{file_tag_list[ii]}: All spectra (outliers in --)\nFinal mean (black), 2x Std Dev (gray, -.)')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')

In [ ]:
wl_nan = proc.wavelengths.copy()
wl_nan[np.logical_not(proc.good_bands_idx)] = np.nan

# Plot without deep water absorption bands
mpl.rcParams['axes.prop_cycle'] = cycler(color=mpl.cm.rainbow(np.linspace(0,1,num=rfl_mat.shape[0])))

line_width = 3
plt.figure()
plt.plot(wl_nan,np.transpose(rfl_mat),linewidth=line_width/2)
plt.plot(wl_nan,np.squeeze(rfl_mean),'k',linewidth=line_width)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance')